In [6]:
import gurobipy as gp
from gurobipy import *

In [7]:
def owt(model, ibfs):
    cct = model.NumConstrs
    constrs = model.getConstrs()
    vct = model.NumVars
    #tableau has 1 column per variable, plus one (leading) for z and one (trailing) for RHS; tableau has 1 row per constraint, plus one (leading) for the objective
    tableau = [[0 for j in range(vct+2)] for i in range(cct+1)]
    vars = model.getVars()
    #objective row
    tableau[0][0] = 1
    for i in range(vct):
        tableau[0][i+1] = -1*vars[i].obj
    #constraint rows
    for i in range(cct):
        for j in range(vct):
            ci = constrs[i]
            vj = vars[j]
            tableau[i+1][j+1] = model.getCoeff(ci, vj)
        tableau[i+1][vct+1] = constrs[i].rhs
    #update based on ibfs
    rowlabels = ['' for i in range(cct+1)]
    rowlabels[0] = 'z'
    for i in range(vct):
        if ibfs[i] == 0:
            continue
        for j in range(1, cct+1):
            if tableau[j][i+1] != 0 and len(rowlabels[j]) == 0:
                rowlabels[j] = vars[i].VarName
                if tableau[j][i+1] != 1:
                    mult = 1/tableau[j][i+1]
                    for i2 in range(vct+2):
                        tableau[j][i2] = mult*tableau[j][i2]
                for j2 in range(1, cct+1):
                    if j2 == j or tableau[j2][i+1] == 0:
                        continue
                    mult = tableau[j2][i+1]
                    for i2 in range(vct+2):
                        tableau[j2][i2] = tableau[j2][i2]-mult*tableau[j][i2]
    while True:
        least = 0
        leasto = -1
        for i in range(1,vct+1):
            if tableau[0][i] < least:
                least = tableau[0][i]
                leasto = i
        if leasto < 0:
            print('optimal value is '+str(tableau[0][vct+1]))
            return (tableau, rowlabels)
        lowrat = 1e20
        lowown = -1
        for i in range(1, cct+1):
            if tableau[i][leasto]*tableau[i][vct+1] <= 0:
                continue
            if tableau[i][vct+1]/tableau[i][leasto] < lowrat:
                lowrat = tableau[i][vct+1]/tableau[i][leasto]
                lowown = i
        rowlabels[lowown] = vars[leasto-1].VarName
        if tableau[lowown][leasto] != 1:
            rat = tableau[lowown][leasto]
            for i in range(vct+2):
                tableau[lowown][i] = tableau[lowown][i]/rat
        for i in range(cct+1):
            if i == lowown or tableau[i][leasto] == 0:
                continue
            mult = tableau[i][leasto]
            for j in range(vct+2):
                tableau[i][j] = tableau[i][j]-mult*tableau[lowown][j]
            
    
    
    
    


In [12]:
#assumes model is maximization in Ax <= b form, x >= 0, b >= 0
from os import rmdir


fname = input('Please input file name correctly:')
m = gp.read(fname)
constrs = m.getConstrs()
slack_var = 0
ibfs = [0 for i in range(m.numVars)]
for c in constrs:
    slack_var += 1
    s = m.addVar(name= 's'+str(slack_var))
    ibfs.append(c.rhs)
    m.addConstr(quicksum(m.getCoeff(c, var)*var for var in m.getVars())+s == c.rhs)
    m.remove(c)
m.update()
m.optimize()
flag = True
tol = 1e-10

cuts = 0
while flag:
    (tableau, rowlabels) = owt(m, ibfs)
    rm = len(rowlabels)
    n = len(tableau[0])
    flag = False
    for i in range(1, rm):
        if tableau[i][-1] % 1 > tol and 1-(tableau[i][-1] % 1) > tol:
            slack_var += 1
            s = m.addVar(name= 's'+str(slack_var))
            m.addConstr(quicksum(-1*(tableau[i][j] % 1)*m.getVars()[j-1] for j in range(1, n-1)) + s == -1*(tableau[i][-1] % 1))
            cuts += 1
            m.update()
            sval = -1*(tableau[i][-1] % 1)
            for j in range(n-2):
                sval += (tableau[i][j+1] % 1)*ibfs[j]
            ibfs.append(sval)
            flag = True
            break

print('Total numbers of cuts:', str(cuts))
print('Optimal value:', str(tableau[0][-1]))

    
    
    



Read MPS format model from file test_model_4.mps
Reading time = 0.00 seconds
Test: 7 rows, 5 columns, 14 nonzeros
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 12 columns and 21 nonzeros
Model fingerprint: 0x57605beb
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 6 rows and 9 columns
Presolve time: 0.01s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e+01   2.200000e+00   0.000000e+00      0s
       1    3.2875000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  3.287500000e+01
optimal value is 32.875
optimal value is 32.86666666666667
optimal value is 32.857142857142854
optimal value is 32.84615384615385
optimal v

31.000000000000007
